# Evaluation

In [ ]:
import os
import sys
sys.path.append('../../src')

%load_ext autoreload
%autoreload 2
%load_ext autotime

from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.constants import k as k_B, h as h_P

k_B /= h_P * 1e9
matplotlib.rcParams.update({"font.size": 14})

from models import sqRBM_em
from models import RBM_em
from utils import (
    Discretizer,
    kl_divergence,
    get_project_dir, # returns: Path object of the project directory.
    get_rng, # returns: Numpy RandomState object.
    lr_exp_decay, # returns: The learning rate scaling factor
    load_params,
)

## Visualization

### Plot results

In [ ]:
# plot one situation

import pickle
import os
import numpy as np
import matplotlib.pyplot as plt

num_situation = 101
time_start = 1
time_end = 10

for time in range(time_start, time_end + 1):

    params_path = f'../data/params/situation{num_situation}.json'
    params = load_params(params_path)
    print(params)
    n_visible = params["n_visible"]
    n_hidden = params["n_hidden"]
    n_qubits = n_visible + n_hidden
    n_epochs = params["n_epochs"]
    n_epochs_m = params["n_epochs_m"]
    learning_rate = params["learning_rate"]
    epsilon_em = params["epsilon_em"]
    epsilon_gd = params["epsilon_gd"]
    # seed = params["seed"]

    load_dir = f'../data/output/results/situation{num_situation}/time{time}'
    names = ['kld_emq_epoch_m', 'kld_emc_epoch_m', 'kld_gdq', 'kld_gdc', 'kld_emq_epoch', 'kld_emc_epoch']
    # names = ['kld_emq', 'kld_emc', 'kld_gdq', 'kld_gdc']
    # names = ['kld_emq_epoch_m', 'kld_emc_epoch_m', 'kld_gdq', 'kld_gdc', 'kld_emq_AGD_epoch_m', 'kld_emc_AGD_epoch_m']

    results = {}
    for name in names:
        filename = f'{name}'
        filepath = os.path.join(load_dir, filename)

        with open(filepath, 'rb') as f:
            results[name] = pickle.load(f)

    epochs = np.arange(0, n_epochs)
    fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=300)

    ax.plot(epochs, results['kld_emq_epoch_m'][:n_epochs], linewidth=1.5, label='RCQBM_em: kld')
    # ax.plot(epochs, results['kld_emq_AGD_epoch_m'][:n_epochs], linewidth=1.5, label='RCQBM_em_AGD: kld')
    ax.plot(epochs, results['kld_gdq'][:n_epochs], linewidth=1.5, label='RCQBM: kld')

    ax.plot(epochs, results['kld_emc_epoch_m'][:n_epochs], linewidth=1.5, label='RBM_em: kld')
    # ax.plot(epochs, results['kld_emc_AGD_epoch_m'][:n_epochs], linewidth=1.5, label='RBM_em_AGD: kld')
    ax.plot(epochs, results['kld_gdc'][:n_epochs], linewidth=1.5, label='RBM: kld')

    ax.set_xlabel("Epoch")
    ax.set_ylabel(r"$D_{KL}(p_{data} \ || \ p_{model})$")
    ax.set_yticks(np.arange(0, 100, 1)/100)
    # ax[1].set_yticks(np.arange(40, 110, 10))
    # ax.set_xlim((0, 200000))
    ax.set_ylim((0, 0.1))
    # ax[1].set_ylim((40, 100))
    ax.grid(alpha=0.7)
    ax.legend(loc="upper right")
    plt.tight_layout()

### Plot average

In [ ]:
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt

def get_data_average(situations):
    results_from_file = {}
    results_average = {}
    for num_situation in situations:
        # names = ['kld_emq_epoch_m', 'kld_emc_epoch_m', 'kld_gdq', 'kld_gdc']
        # names = ['kld_emq_epoch_m', 'kld_gdq']
        names = ['kld_emq_epoch_m', 'kld_emc_epoch_m']
        # names = ['kld_emq_epoch_m']
        # names = ['kld_emq', 'kld_emc', 'kld_gdq', 'kld_gdc']
        # names = ['kld_emq_epoch_m', 'kld_emc_epoch_m', 'kld_gdq', 'kld_gdc', 'kld_emq_AGD_epoch_m', 'kld_emc_AGD_epoch_m']
        for name in names:
            results_times = np.zeros((500000, 100))
            for time in range(1, 100 + 1):
                load_dir = f'../data/output/results/situation{num_situation}/time{time}'
                filename = f'{name}'
                filepath = os.path.join(load_dir, filename)

                with open(filepath, 'rb') as f:
                    results_from_file[f'{name}_{num_situation}_{time}'] = pickle.load(f)

                for i in range(0, 500000):
                    results_times[i][time-1] += results_from_file[f'{name}_{num_situation}_{time}'][i]

            results_average[f'{name}_{num_situation}'] = np.average(results_times, axis=1)

    return results_average

def get_data(situations):
    results_from_file = {}
    results_average = {}
    results_std = {}
    results_max = {}
    results_min = {}
    for num_situation in situations:
        # names = ['kld_emq_epoch_m', 'kld_emc_epoch_m', 'kld_gdq', 'kld_gdc']
        names = ['kld_emq_epoch_m', 'kld_gdq']
        # names = ['kld_emq_epoch_m']
        # names = ['kld_emq', 'kld_emc', 'kld_gdq', 'kld_gdc']
        # names = ['kld_emq_epoch_m', 'kld_emc_epoch_m', 'kld_gdq', 'kld_gdc', 'kld_emq_AGD_epoch_m', 'kld_emc_AGD_epoch_m']
        for name in names:
            results_times = np.zeros((500000, 100))
            for time in range(1, 100 + 1):
                load_dir = f'../data/output/results/situation{num_situation}/time{time}'
                filename = f'{name}'
                filepath = os.path.join(load_dir, filename)

                with open(filepath, 'rb') as f:
                    results_from_file[f'{name}_{num_situation}_{time}'] = pickle.load(f)

                for i in range(0, 500000):
                # results_times = []
                    results_times[i][time-1] += results_from_file[f'{name}_{num_situation}_{time}'][i]

            results_average[f'{name}_{num_situation}'] = np.average(results_times, axis=1)
            # results_std[f'{name}_{num_situation}'] = np.std(results_times, axis=1)
            results_max[f'{name}_{num_situation}'] = np.max(results_times, axis=1)
            results_min[f'{name}_{num_situation}'] = np.min(results_times, axis=1)

    # return results_average, results_std
    return results_average, results_max, results_min

def get_data_emq(situations):
    results_from_file = {}
    results_average = {}
    results_std = {}
    # results_max = {}
    # results_min = {}
    for num_situation in situations:
        # names = ['kld_emq_epoch_m', 'kld_emc_epoch_m', 'kld_gdq', 'kld_gdc']
        # names = ['kld_emq_epoch_m', 'kld_gdq']
        names = ['kld_emq_epoch_m']
        # names = ['kld_emq', 'kld_emc', 'kld_gdq', 'kld_gdc']
        # names = ['kld_emq_epoch_m', 'kld_emc_epoch_m', 'kld_gdq', 'kld_gdc', 'kld_emq_AGD_epoch_m', 'kld_emc_AGD_epoch_m']
        for name in names:
            results_times = np.zeros((500000, 100))
            for time in range(1, 100 + 1):
                load_dir = f'../data/output/results/situation{num_situation}/time{time}'
                filename = f'{name}'
                filepath = os.path.join(load_dir, filename)

                with open(filepath, 'rb') as f:
                    results_from_file[f'{name}_{num_situation}_{time}'] = pickle.load(f)

                for i in range(0, 500000):
                # results_times = []
                    results_times[i][time-1] += results_from_file[f'{name}_{num_situation}_{time}'][i]

            results_average[f'{name}_{num_situation}'] = np.average(results_times, axis=1)
            results_std[f'{name}_{num_situation}'] = np.std(results_times, axis=1)
            # results_max[f'{name}_{num_situation}'] = np.max(results_times, axis=1)
            # results_min[f'{name}_{num_situation}'] = np.min(results_times, axis=1)

    return results_average, results_std
    # return results_average, results_max, results_min

def get_data_gdq(situations):
    results_from_file = {}
    results_average = {}
    results_std = {}
    # results_max = {}
    # results_min = {}
    for num_situation in situations:
        # names = ['kld_emq_epoch_m', 'kld_emc_epoch_m', 'kld_gdq', 'kld_gdc']
        # names = ['kld_emq_epoch_m', 'kld_gdq']
        # names = ['kld_emq_epoch_m']
        names = ['kld_gdq']
        # names = ['kld_emq', 'kld_emc', 'kld_gdq', 'kld_gdc']
        # names = ['kld_emq_epoch_m', 'kld_emc_epoch_m', 'kld_gdq', 'kld_gdc', 'kld_emq_AGD_epoch_m', 'kld_emc_AGD_epoch_m']
        for name in names:
            results_times = np.zeros((500000, 100))
            for time in range(1, 100 + 1):
                load_dir = f'../data/output/results/situation{num_situation}/time{time}'
                filename = f'{name}'
                filepath = os.path.join(load_dir, filename)

                with open(filepath, 'rb') as f:
                    results_from_file[f'{name}_{num_situation}_{time}'] = pickle.load(f)

                for i in range(0, 500000):
                # results_times = []
                    results_times[i][time-1] += results_from_file[f'{name}_{num_situation}_{time}'][i]

            results_average[f'{name}_{num_situation}'] = np.average(results_times, axis=1)
            results_std[f'{name}_{num_situation}'] = np.std(results_times, axis=1)
            # results_max[f'{name}_{num_situation}'] = np.max(results_times, axis=1)
            # results_min[f'{name}_{num_situation}'] = np.min(results_times, axis=1)

    return results_average, results_std
    # return results_average, results_max, results_min

In [ ]:
# select situations to culculate average results

situations = [101, 102, 103, 104, 105, 106, 107, 108, 109]
# situations = [101, 102, 103]
# situations = [104, 105, 106]
# situations = [107, 108, 109]

# situations = [110, 111, 112, 113, 114, 115, 116, 117, 118]
# situations = [110, 111, 112]
# situations = [113, 114, 115]
# situations = [116, 117, 118]

# situations = [119, 120, 121, 122, 123, 124, 125, 126, 127]
# situations = [119, 120, 121]
# situations = [122, 123, 124]
# situations = [125, 126, 127]

# situations = [128, 129, 130, 131, 132, 133, 134, 135, 136]
# situations = [128, 129, 130]
# situations = [131, 132, 133]
# situations = [134, 135, 136]

# results_average, results_std = get_data(situations)
# results_average, results_max, results_min = get_data(situations)
results_average = get_data_average(situations)

In [ ]:
# plot average results for each situation

names_colors = [[('kld_emq_epoch_m', 'blue', 'o', 'em_quantum, M=3'), ('kld_gdq', 'red', 'o', 'gd_quantum, M=3')],
                [('kld_emq_epoch_m', 'blue', '^', 'em_quantum, M=4'), ('kld_gdq', 'red', '^', 'gd_quantum, M=4')],
                [('kld_emq_epoch_m', 'blue', 's', 'em_quantum, M=5'), ('kld_gdq', 'red', 's', 'gd_quantum, M=5')]]

# names_colors = [[('kld_emq_epoch_m', 'blue', 'o', 'em_quantum, M=3'), ('kld_emc_epoch_m', 'green', 'o', 'gd_quantum, M=3')],
#                 [('kld_emq_epoch_m', 'blue', '^', 'em_quantum, M=4'), ('kld_emc_epoch_m', 'green', '^', 'gd_quantum, M=4')],
#                 [('kld_emq_epoch_m', 'blue', 's', 'em_quantum, M=5'), ('kld_emc_epoch_m', 'green', 's', 'gd_quantum, M=5')]]

# situations = [101]
situations = [101, 102, 103]
# situations = [104, 105, 106]
# situations = [107, 108, 109]

# situations = [110, 111, 112]
# situations = [113, 114, 115]
# situations = [116, 117, 118]

# situations = [119, 120, 121]
# situations = [122, 123, 124]
# situations = [125, 126, 127]

# situations = [128, 129, 130]
# situations = [131, 132, 133]
# situations = [134, 135, 136]


n_epochs = 500000
# for num_situation in situations:
epochs = np.arange(0, n_epochs)
fig, ax = plt.subplots(1, 1, figsize=(4, 3), dpi=300)
for i in range(len(situations)):
    num_situation = situations[i]
    print('num_situation', num_situation)
    # for name, color in names_colors[i].items():
    for name, color, marker, label in names_colors[i]:
        ax.plot(epochs, results_average[f'{name}_{num_situation}'], marker=f'{marker}', ms=3, markevery=50000, linewidth=1, label=f'{label}', color=color)
        # ax.plot(epochs, results_average[f'{name}_{num_situation}'], marker=f'{marker}', ms=5, markevery=2000, linewidth=1, label=f'{label}', color=color)
        # ax.fill_between(epochs, results_average[f'{name}_{num_situation}'] - results_std[f'{name}_{num_situation}'], results_average[f'{name}_{num_situation}'] + results_std[f'{name}_{num_situation}'], color=color, alpha=0.3, edgecolor='none',)
        # ax.plot(epochs, results_max[f'{name}_{num_situation}'], linewidth=1, label=f'{name}_{num_situation}_max', color=color, linestyle=':')
        # ax.plot(epochs, results_max[f'{name}_{num_situation}'], linewidth=0.5, color=color, linestyle=':')
        # ax.plot(epochs, results_min[f'{name}_{num_situation}'], linewidth=1, label=f'{name}_{num_situation}_min', color=color, linestyle=':')
        # ax.plot(epochs, results_min[f'{name}_{num_situation}'], linewidth=0.5, color=color, linestyle=':')

# ax.set_xlabel("Epoch")
# ax.set_ylabel(r"$D_{KL}(p_{data} \ || \ p_{model})$")
ax.set_xticks(np.arange(0, 500001, 5000))
# ax.set_yticks(np.arange(0, 100, 1)/10)
# ax[1].set_yticks(np.arange(40, 110, 10))
ax.set_xlim((0, 500000))
ax.set_ylim((0, 1.0))
# ax.set_title(f'situation{num_situation}')
# ax[1].set_ylim((40, 100))
ax.grid(alpha=0.7)
ax.tick_params(labelsize=10)
# ax.legend(loc="upper right", fontsize=8)
plt.tight_layout()
plt.show()

### Make legend

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# labels = ['em, M=3', 'gd, M=3', 'em, M=4',
#           'gd, M=4', 'em, M=5', 'gd, M=5']
# labels = ['em, M=3',  'em, M=4', 'em, M=5',
#            'gd, M=3', 'gd, M=4', 'gd, M=5']
labels = ['em_quantum, M=3',  'em_quantum, M=4', 'em_quantum, M=5',
           'em_classical, M=3', 'em_classical, M=4', 'em_classical, M=5']
# labels = ['em_quantum, M=3', 'em_classical, M=3', 'em_quantum, M=4', 
#            'em_classical, M=4', 'em_quantum, M=5', 'em_classical, M=5']

# names_colors = [[('kld_emq_epoch_m', 'blue', 'o', 'em_quantum, M=3'), ('kld_emc_epoch_m', 'green', '^', 'gd_quantum, M=3')],
#                 [('kld_emq_epoch_m', 'green', 'o', 'em_quantum, M=4'), ('kld_emc_epoch_m', 'blue', 's', 'gd_quantum, M=4')],
#                 [('kld_emq_epoch_m', 'blue', '^', 'em_quantum, M=5'), ('kld_emc_epoch_m', 'green', 's', 'gd_quantum, M=5')]]

fig_leg = plt.figure(figsize=(6.0, 0.2))

# handles = [Line2D([], [], color=names_colors[i % 3][j % 2][1], marker=names_colors[i % 3][j % 2][2], lw=2)
#            for i in range(len(labels)) for j in range(len(labels))]
# handles = [Line2D([], [], color=names_colors[i // 2][i % 2][1], marker=names_colors[i // 2][i % 2][2], ms=12, lw=3)
#            for i in range(len(labels))]
handles = [Line2D([], [], color=names_colors[i % 3][0 if i<3 else 1][1], marker=names_colors[i % 3][0 if i<3 else 1][2], ms=12, lw=3)
           for i in range(len(labels))]

print([([0 if i<3 else 1][i % 2] for i in range(len(labels)))])

fig_leg.legend(handles, labels,
               loc='center',
               frameon=False,
               ncol=6,
               fontsize=20,
               handlelength=1.5,
               handletextpad=0.4,
               columnspacing=0.8,
               labelspacing=0.6)

fig_leg.tight_layout(pad=0)
fig_leg.gca().axis('off')

plt.show()

### Print average

In [ ]:
# print average results for each situation

import pickle
import os

situations = []

for num_situation in situations:
    average_emq = 0
    average_emc = 0
    average_gdq = 0
    average_gdc = 0
    for time in range(1, 100 + 1):
        load_dir = f'../data/output/results/situation{num_situation}/time{time}'
        names = ['kld_emq_epoch_m', 'kld_emc_epoch_m', 'kld_gdq', 'kld_gdc', 'kld_emq_epoch', 'kld_emc_epoch']
        # names = ['kld_emq', 'kld_emc', 'kld_gdq', 'kld_gdc']
        # names = ['kld_emq_epoch_m', 'kld_emc_epoch_m', 'kld_gdq', 'kld_gdc', 'kld_emq_AGD_epoch_m', 'kld_emc_AGD_epoch_m']

        results_average = {}
        for name in names:
            filename = f'{name}'
            filepath = os.path.join(load_dir, filename)

            with open(filepath, 'rb') as f:
                results_average[name] = pickle.load(f)

        average_emq += results_average['kld_emq_epoch_m'][500000-1]
        average_emc += results_average['kld_emc_epoch_m'][500000-1]
        average_gdq += results_average['kld_gdq'][500000-1]
        average_gdc += results_average['kld_gdc'][500000-1]

    # print('time', time)
    print(f'situation{num_situation}')
    print('average_emq', average_emq/100)
    print('average_emc', average_emc/100)
    print('average_gdq', average_gdq/100)
    print('average_gdc', average_gdc/100)